In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd
import numpy as np
import os

def clean_and_process_data(input_file, output_file):
    print(f"Loading data from {input_file}...")

    try:
        df = pd.read_csv(input_file, low_memory=False)
    except FileNotFoundError:
        print(f"Error: The file '{input_file}' was not found.")
        print("Please make sure the CSV file is in the same folder as this script.")
        return

    # 1. Drop unnecessary PII and administrative columns
    # We remove high-cardinality text (Job Title, Skills) that requires NLP, keeping structured data.
    cols_to_drop = [
        'EMP_CONTACT_NAME', 'EMP_CONTACT_CITY', 'EMP_CONTACT_STATE_PROVINCE',
        'SPECIFIC_SKILLS', 'EMPLOYER_NAME', 'EMPLOYER_CITY', 'WORKSITE_CITY',
        'JOB_TITLE', 'MAJOR_FIELD_OF_STUDY', 'CASE_NUMBER'
    ]
    df_clean = df.drop(columns=cols_to_drop, errors='ignore')

    # 2. Date Conversion
    print("Processing dates...")
    df_clean['RECEIVED_DATE'] = pd.to_datetime(df_clean['RECEIVED_DATE'], errors='coerce')
    df_clean['DECISION_DATE'] = pd.to_datetime(df_clean['DECISION_DATE'], errors='coerce')

    # Drop rows where critical dates are missing
    df_clean = df_clean.dropna(subset=['RECEIVED_DATE', 'DECISION_DATE'])

    # 3. Calculate Target Variable: Processing Time (Days)
    df_clean['PROCESSING_TIME_DAYS'] = (df_clean['DECISION_DATE'] - df_clean['RECEIVED_DATE']).dt.days

    # Remove negative processing times (data errors)
    df_clean = df_clean[df_clean['PROCESSING_TIME_DAYS'] >= 0]

    # 4. Clean Wage Data
    print("Cleaning wage data...")
    def clean_wage(wage):
        if pd.isna(wage):
            return np.nan
        if isinstance(wage, str):
            clean_str = wage.replace('$', '').replace(',', '').strip()
            try:
                return float(clean_str)
            except ValueError:
                return np.nan
        return float(wage)

    df_clean['PW_WAGE'] = df_clean['PW_WAGE'].apply(clean_wage)

    # 5. Handle Missing Values (Imputation)
    fill_values = {
        'PW_SKILL_LEVEL': 'Unknown',
        'MINIMUM_EDUCATION': 'Unknown',
        'PW_SOC_TITLE': 'Unknown',
        'WORKSITE_STATE': 'Unknown',
        'EMPLOYER_STATE_PROVINCE': 'Unknown'
    }
    df_clean = df_clean.fillna(value=fill_values)

    # 6. Feature Engineering
    print("Engineering features...")
    # Seasonality
    df_clean['SUBMISSION_MONTH'] = df_clean['RECEIVED_DATE'].dt.month
    df_clean['SUBMISSION_YEAR'] = df_clean['RECEIVED_DATE'].dt.year

    # Job Category Grouping (First 2 digits of SOC code)
    df_clean['SOC_MAJOR_GROUP'] = df_clean['PW_SOC_CODE'].astype(str).str[:2]

    # 7. Final Column Selection
    # We remove DECISION_DATE to prevent data leakage in the model
    final_cols = [
        'RECEIVED_DATE',
        'EMPLOYER_STATE_PROVINCE',
        'PW_SOC_CODE',
        'PW_SOC_TITLE',
        'SOC_MAJOR_GROUP',
        'PW_SKILL_LEVEL',
        'PW_WAGE',
        'WORKSITE_STATE',
        'MINIMUM_EDUCATION',
        'SUBMISSION_MONTH',
        'SUBMISSION_YEAR',
        'CASE_STATUS',          # Target for Classification
        'PROCESSING_TIME_DAYS'  # Target for Regression
    ]

    # Ensure all columns exist
    available_cols = [c for c in final_cols if c in df_clean.columns]
    df_final = df_clean[available_cols]

    # 8. Save Output
    print(f"Saving processed data to {output_file}...")
    df_final.to_csv(output_file, index=False)
    print("Success! Data processing complete.")
    print(f"Rows processed: {len(df_final)}")

if __name__ == "__main__":
    # Updated input path as requested
    input_csv = '/content/drive/MyDrive/Datasets/PERM_Disclosure_Data_FY2025_Q3_Final.csv'
    output_csv = 'Processed_Visa_Dataset.csv'

    clean_and_process_data(input_csv, output_csv)

Loading data from /content/drive/MyDrive/Datasets/PERM_Disclosure_Data_FY2025_Q3_Final.csv...
Processing dates...
Cleaning wage data...
Engineering features...
Saving processed data to Processed_Visa_Dataset.csv...
Success! Data processing complete.
Rows processed: 1171


In [6]:
from google.colab import files
import os

# Define the file path
file_path = 'Processed_Visa_Dataset.csv'

# Check if the file exists to prevent errors, then download
if os.path.exists(file_path):
    print(f"Found {file_path}. Starting download...")
    files.download(file_path)
else:
    print(f"Error: {file_path} not found. Please ensure the processing script finished successfully.")

Found Processed_Visa_Dataset.csv. Starting download...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [7]:
import pandas as pd
# Load the processed CSV file
output_csv = 'Processed_Visa_Dataset.csv'
df_processed = pd.read_csv(output_csv)

# Display dataset dimensions
print(f"Dataset Shape: {df_processed.shape}")
print("-" * 30)

# Display the first 5 rows
print("First 5 rows of the processed dataset:")
# In Jupyter/Colab, simply typing the dataframe variable at the end displays it nicely
# If you are using a script, use print(df_processed.head())
print(df_processed.head())

Dataset Shape: (1171, 13)
------------------------------
First 5 rows of the processed dataset:
  RECEIVED_DATE EMPLOYER_STATE_PROVINCE PW_SOC_CODE  \
0    2023-02-27                ILLINOIS  25-1067.00   
1    2022-10-03          NORTH CAROLINA     11-3031   
2    2022-07-22          CONNECTICUT CT     11-9051   
3    2022-06-20                 FLORIDA     11-9013   
4    2022-09-30                 GEORGIA     11-9021   

                        PW_SOC_TITLE SOC_MAJOR_GROUP PW_SKILL_LEVEL   PW_WAGE  \
0  Sociology Teachers, Postsecondary              25        Level I   44490.0   
1                 Financial Managers              11        Level I  125986.0   
2              Food Service Managers              11       Level IV  119434.0   
3              Regional Farm Manager              11       Level IV  161866.0   
4              Construction Managers              11       Level II   83762.0   

  WORKSITE_STATE MINIMUM_EDUCATION  SUBMISSION_MONTH  SUBMISSION_YEAR  \
0       ILLIN